In [56]:
using LinearAlgebra, Test, Plots, Random, Distributions, TensorToolbox, Arpack
pyplot()
nothing;

In [57]:
function modecontract(S::Array{T,N}, k::Int, Z::Matrix{T}) where {T<:Number,N}
    @assert k ∈ 1:N
    n = collect(size(S))
    @assert size(S, k) == n[k]
    p = prod(n[1:k-1])
    q = prod(n[k+1:N])
    S = reshape(S, p, n[k], q)
    S = permutedims(S, [2,1,3])
    S = reshape(S, n[k], p*q)
    S = Z*S
    n[k] = size(S, 1)
    S = reshape(S, n[k], p, q)
    S = permutedims(S, [2,1,3])
    S = reshape(S, n...)
    
    return S
end
nothing;


In [58]:
function unfolding(A::Array{T,N},k::Int) where {T<:Number,N}
    @assert k ∈ 1:N
    S=copy(A)
    n = collect(size(S))
    @assert size(S, k) == n[k]
    p = prod(n[1:k-1])
    q = prod(n[k+1:N])
    S = reshape(S, p, n[k], q)
    S = permutedims(S, [1,3,2])
    S = reshape(S,p*q,n[k])
    return S
end

unfolding (generic function with 1 method)

In [59]:
function HOSVD(A, reqrank)

    N = ndims(A)
    S = copy(A)
    Uk = []
    for k = 1:N
        n = collect(size(S))               #size alpha1,...,alphak-1,ik,ik+1,...,id
        p = prod(n[1:k-1])
        q = prod(n[k+1:N])
        
        Bk = unfolding(copy(S), k)
        Bk_svd = svd(Bk)
                                        
        U1 = Bk_svd.U[ :, 1:reqrank[k] ]                      #generating approximation
        V1t = Bk_svd.Vt[1:reqrank[k],: ]
        Sigma1 = Diagonal( Bk_svd.S[1:reqrank[k]] )
        
        Wk =  Bk*V1t'  #generating Wk
        B_hat = Wk*V1t
        @show norm(V1t*V1t')
        S=Wk   
        
        n[k] = size(S, 2)      
        S = reshape(S, p, q,n[k])                          #reshape back to tensor form
        S = permutedims(S, [1,3,2])
        S = reshape(S, n...)
        V1= permutedims(V1t,[2,1])
        push!(Uk, V1)
        

    end
    X=S   
    for k=1:N
        X = modecontract(X, k, Uk[k])
    end
    return X,Uk
end

HOSVD (generic function with 1 method)

In [60]:
T = Float64
d=4
n = [20+i for i in 1:d]
r = [2*i for i in 1:d]


U=[rand(Uniform(-1,1),n[i],r[i]) for i in 1:d]
S=rand(Uniform(-1,1),r...)

2×4×6×8 Array{Float64, 4}:
[:, :, 1, 1] =
 -0.899819  0.0889598  -0.286566  0.854158
 -0.307115  0.50992    -0.12675   0.271726

[:, :, 2, 1] =
 0.614693  -0.444793  0.189229  -0.41575
 0.554191   0.133267  0.262951  -0.555251

[:, :, 3, 1] =
  0.928478    0.57196   -0.170243   0.219901
 -0.0820648  -0.496119  -0.923549  -0.595463

[:, :, 4, 1] =
 -0.378276  -0.129602  0.496067  -0.0885961
 -0.573169  -0.500557  0.885992   0.881086

[:, :, 5, 1] =
 0.525827  -0.75799   -0.517048  0.642763
 0.612544   0.570724  -0.981089  0.0633887

[:, :, 6, 1] =
 -0.954386   0.755165  0.525377  -0.367522
 -0.845492  -0.640063  0.753058  -0.558725

[:, :, 1, 2] =
 -0.896925   0.415512  -0.630709  -0.763255
 -0.0543993  0.431633   0.487051   0.256885

[:, :, 2, 2] =
 0.958152   0.793018   0.236317   0.379079
 0.189401  -0.0474949  0.276347  -0.477509

[:, :, 3, 2] =
  0.0693223  -0.976794    0.823466    -0.011346
 -0.721779   -0.0639004  -0.00890835   0.392461

[:, :, 4, 2] =
  0.17645    0.321859   0.2

In [61]:
C=copy(S)
for i in 1:d
    C=modecontract(C,i,U[i])
end

C

21×22×23×24 Array{Float64, 4}:
[:, :, 1, 1] =
 -0.0921879  -1.10627    1.58935     -0.783788  …  -0.803221    -0.483421
 -0.197379   -1.14813    1.15847     -0.798354     -0.593212    -0.155261
  0.0645319   0.504695  -0.616572     0.354238      0.313314     0.143982
  0.153512    0.315207   0.161501     0.20444      -0.0719193   -0.295723
 -0.0617102   0.454588  -1.13393      0.336851      0.565471     0.537908
  0.306488    0.603233   0.370403     0.389366  …  -0.167662    -0.609215
  0.0941893   0.242743   0.00834875   0.161006      0.00141838  -0.145876
  0.0516518   0.755264  -1.13956      0.536774      0.575043     0.368481
  0.14957     0.494037  -0.186398     0.333936      0.102464    -0.153386
 -0.161649   -0.989234   1.03876     -0.689112     -0.530999    -0.162433
  0.180822    0.368474   0.195401     0.238785  …  -0.0873083   -0.350359
 -0.129727   -0.745721   0.745055    -0.51831      -0.381684    -0.095639
 -0.109562   -1.01428    1.3363      -0.714897     -0.677241    -0

In [62]:
println(norm(C,2))
C_approx,V = HOSVD(C,r)
nothing;


650.1394953191988
norm(V1t * V1t') = 1.4142135623730958
norm(V1t * V1t') = 1.9999999999999991
norm(V1t * V1t') = 2.4494897427831774
norm(V1t * V1t') = 2.828427124746189


Matrix V isn't orthogonal for some reason

In [63]:
println(norm(C-C_approx)\norm(C))

7.511402208342158e14


I gave up

In [64]:
V[2]

22×4 Matrix{Float64}:
  0.0372281    0.308213    -0.0442742   0.0247871
 -0.330347     0.179087     0.0796319  -0.311092
 -0.216976    -0.0529482    0.267639    0.44445
 -0.00699659   0.394765    -0.023872   -0.131352
 -0.219434     0.387077    -0.368646    0.0337671
 -0.0351702    0.0411288    0.0646817   0.363371
 -0.208397    -0.159229    -0.233159   -0.13813
  0.194353    -0.103081    -0.130454   -0.211826
 -0.0113517   -0.26879      0.0459881  -0.488334
 -0.230812    -0.258642    -0.199445    0.0441701
  0.0354532   -0.275578     0.287213   -0.205846
  0.188637     0.0265287    0.16846     0.00349237
 -0.314281    -0.0205762    0.158063    0.0958983
  0.440389    -0.100836     0.0018504   0.070246
  0.248517     0.361467     0.0187763   0.077009
 -0.069907     0.0963814    0.48232     0.0277382
  0.0292008    0.281776     0.0673084  -0.337761
  0.368157    -0.0241       0.0387291   0.0534517
  0.119699     0.224467     0.14061    -0.219527
 -0.152925     0.148967     0.460525   -0

In [65]:
U[2]

22×4 Matrix{Float64}:
  0.657521    0.585776     0.0259287  -0.00308701
  0.874561   -0.650942     0.0274004   0.759771
  0.110627   -0.803681     0.934023   -0.7371
  0.977357    0.53101     -0.177276    0.230082
  0.744882    0.721514     0.858248    0.966704
  0.0427887   0.00182599   0.640948   -0.535068
 -0.378235   -0.299854     0.384078    0.917409
 -0.431604    0.391761    -0.623689    0.217212
 -0.401605   -0.638037    -0.885185    0.561949
 -0.638609   -0.477784     0.70906     0.652127
 -0.34443    -0.858554    -0.718067   -0.315285
  0.0700976   0.137354    -0.522806   -0.596917
  0.298566   -0.889661     0.622455    0.0621115
 -0.604654    0.764464    -0.737472   -0.761754
  0.659367    0.997171    -0.35218    -0.504559
  0.71112    -0.797144    -0.281678   -0.809366
  0.842763    0.225462    -0.700756    0.274652
 -0.325921    0.659429    -0.659594   -0.696986
  0.657047    0.245598    -0.743544   -0.150019
  0.92039    -0.888922    -0.275847   -0.51079
  0.0982725  -0.69

In [66]:
k=2
S=copy(C)
n = collect(size(S))               #size alpha1,...,alphak-1,ik,ik+1,...,id
p = prod(n[1:k-1])
q = prod(n[k+1:d])

B=unfolding(copy(S),k)
S=B
S = reshape(S, p, q,n[k])                          #reshape back to tensor form
       
S = permutedims(S, [1,3,2])
S = reshape(S, n...)
print(norm(C-S,2))

0.0